In [ ]:
# 1. 
from math import ceil


N = 1024
grids = 8
threadsInBlock = 128

warpsPerBlock = ceil(128 // 32)
print(f"1.a\t{warpsPerBlock} warpsPerBlock")
print(f"1.b\t{grids * warpsPerBlock} warpsInGrid")
print(f"1.c.1\t{3 * grids} Active Warps in Grid, in Line 04 divergence")
print(f"1.c.2\t{2 * grids} Warps divergent in grid, in Line 04 divergence")
print(f"1.c.3\t{32/32*100}% SIMD efficiency warp 0 of block 0 , in Line 04 divergence")
print(f"1.c.4\t{(32-(64-40))/32*100}% SIMD efficiency warp 1 of block 0 , in Line 04 divergence")
print(f"1.c.4\t{(128-104)/32*100}% SIMD efficiency warp 3 of block 0 , in Line 04 divergence")
print(f"1.d.1\t{grids * warpsPerBlock} Warps in grid Active, Line 07 divergence")
print(f"1.d.2\t{grids * warpsPerBlock } Warps in grid Divergent, Line 07 divergence")
print(f"1.d.2\t{50/100*100}% SIMD efficiency warp 0 block 0, Line 07 divergence")
# Loop condition: j: 0 to < 5 - i%3
print(f"1.e.1\t{3} Itrations have NO divergence, Line 09 divergence")
print(f"1.e.1\t{2} Itrations HAVE divergence, Line 09 divergence")

1.a	4 warpsPerBlock
1.b	32 warpsInGrid
1.c.1	24 Active Warps in Grid, in Line 04 divergence
1.c.2	16 Warps divergent in grid, in Line 04 divergence
1.c.3	100.0% SIMD efficiency warp 0 of block 0 , in Line 04 divergence
1.c.4	25.0% SIMD efficiency warp 1 of block 0 , in Line 04 divergence
1.c.4	75.0% SIMD efficiency warp 3 of block 0 , in Line 04 divergence
1.d.1	32 Warps in grid Active, Line 07 divergence
1.d.2	32 Warps in grid Divergent, Line 07 divergence
1.d.2	50.0% SIMD efficiency warp 0 block 0, Line 07 divergence
1.e.1	3 Itrations have NO divergence, Line 09 divergence
1.e.1	2 Itrations HAVE divergence, Line 09 divergence


In [51]:
#2. For a vector addition, assume that the vector length is 2000, each thread
# calculates one output element, and the thread block size is 512 threads. How
# many threads will be in the grid?

from atexit import register


print(f"2.\t{512*4} threads in grid")
#3. For the previous question, how many warps do you expect to have divergence
# due to the boundary check on vector length?
print(f"3.\t{1} threads divergent")
# 4. Consider a hypothetical block with 8 threads executing a section of code
# before reaching a barrier. The threads require the following amount of time
# (in microseconds) to execute the sections: 2.0, 2.3, 3.0, 2.8, 2.4, 1.9, 2.6, and
# 2.9; they spend the rest of their time waiting for the barrier. What percentage
# of the threads’ total execution time is spent waiting for the barrier?
times = [2.0, 2.3, 3.0, 2.8, 2.4, 1.9, 2.6, 2.9]
waitTimePercentage = sum(max(times) - t for t in times) / (8 * max(times)) * 100
print(f"4.\t{waitTimePercentage:.2f}% of threads execution time waiting for barrier")
# 5. A CUDA programmer says that if they launch a kernel with only 32 threads
# in each block, they can leave out the __syncthreads() instruction wherever
# barrier synchronization is needed. Do you think this is a good idea? Explain.
print(f"5.\tNo. Warp size can change across architectures in future. __syncthreads() sync within the same block. Barrier for within blocks")
# 6. If a CUDA device’s SM can take up to 1536 threads and up to 4 thread
# blocks, which of the following block configurations would result in the most
# number of threads in the SM?
# a. 128t/b, 256t/b, 512t/b, 1024t/b
print(f"6.\t{512}threads per block can fit 3 blocks in the SM, max is 4")
# 7. Assume a device that allows up to 64 blocks per SM and 2048 threads per
# SM. Indicate which of the following assignments per SM are possible. In the
# cases in which it is possible, indicate the occupancy level.
totalThreads = 2048
blocks = [8,16,32,64,32]
threads = [128,64,32,32,64]
problemCh = 'a'
for b,t in zip(blocks, threads):
  threadsInSM = b * t
  occupancy = threadsInSM / totalThreads * 100
  print(f"7.{problemCh}\t{occupancy:.2f}% occupancy{b}: blocks of {t} threads")
  problemCh = chr(ord(problemCh)+1)

# 8. Consider a GPU with the following hardware limits: 2048 threads per SM, 32
# blocks per SM, and 64K (65,536) registers per SM. For each of the following
# kernel characteristics, specify whether the kernel can achieve full occupancy.
# If not, specify the limiting factor.
# a. The kernel uses 128 threads per block and 30 registers per thread.
# b. The kernel uses 32 threads per block and 29 registers per thread.
# c. The kernel uses 256 threads per block and 34 registers per thread.
threadsPerSM = 2048
blocksPerSM = 32
registersPerSM = 65536
problemCh = 'a'
threads = [128,32,256]
regsPerThread = [30,29,34]
print(f"8.a\t Full occupancy can be achieved")
print(f"8.b\t Occupancy limited by #Blocks")
print(f"8.c\t Occupancy limited by registers per SM")

print(f"9.\tBluff. SM supports 512 threads. A 32x32 thread block kernel calculating for 1024x1024 element array requires 32x32=1024 threads per block, when limit is 512 threads per block")


2.	2048 threads in grid
3.	1 threads divergent
4.	17.08% of threads execution time waiting for barrier
5.	No. Warp size can change across architectures in future. __syncthreads() sync within the same block. Barrier for within blocks
6.	512threads per block can fit 3 blocks in the SM, max is 4
7.a	50.00% occupancy8: blocks of 128 threads
7.b	50.00% occupancy16: blocks of 64 threads
7.c	50.00% occupancy32: blocks of 32 threads
7.d	100.00% occupancy64: blocks of 32 threads
7.e	100.00% occupancy32: blocks of 64 threads
8.a	 Full occupancy can be achieved
8.b	 Occupancy limited by #Blocks
8.c	 Occupancy limited by registers per SM
9.	Bluff. SM supports 512 threads. A 32x32 thread block kernel calculating for 1024x1024 element array requires 32x32=1024 threads per block, when limit is 512 threads per block
